In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import xgboost as xgb

datapath = r'./Regression/data'

## Slice Localization data split

In [11]:
# Load the data: slice_localization
df = pd.read_csv(datapath+'/datwTestTrainSplit.csv')

In [12]:
df.head(5)

,Unnamed: 0,claim_no,claim_size,occurrence_period,occurrence_time,noti_period,notidel,setldel,settle_period,development_period,...,is_settled,payment_size,pmt_no,trn_no,payment_size_cumulative,log1_paid_cumulative,paid_dev_factor,max_paid_dev_factor,min_paid_dev_factor,train_ind
0,0,1,232310.088225,1,0.734094,2,0.658094,23.214926,25,1,...,False,0.000000,0.0,1.0,0.000000,0.000000,NaN,0.0,0.0,False
1,1,1,232310.088225,1,0.734094,2,0.658094,23.214926,25,2,...,False,0.000000,0.0,1.0,0.000000,0.000000,NaN,0.0,0.0,False
2,2,1,232310.088225,1,0.734094,2,0.658094,23.214926,25,3,...,False,0.000000,0.0,1.0,0.000000,0.000000,NaN,0.0,0.0,False
3,3,1,232310.088225,1,0.734094,2,0.658094,23.214926,25,4,...,False,0.000000,0.0,1.0,0.000000,0.000000,NaN,0.0,0.0,False
4,4,1,232310.088225,1,0.734094,2,0.658094,23.214926,25,5,...,False,13226.339829,1.0,2.0,13226.339829,9.490041,NaN,0.0,0.0,False


In [17]:
# Slice data train and test split: Patient id shouldn't get mixed during train and test split
## u_pid = np.unique(df['patientId'])
## print(u_pid, len(u_pid))

# 80 - 20% split based on patient id: 
## test_idx = np.arange(1, int(len(u_pid)/5) + 1)*5 - 1
## print(test_idx)
train = np.where(df.train_ind)[0]
test = np.where(~df.train_ind)[0]
## test = np.where(np.isin(df['patientId'],test_idx))[0]
## train = np.where(~np.isin(df['patientId'],test_idx))[0]
print(len(test), len(train))

57228 85613


In [18]:
list_of_features = [
    "claim_no",
    "occurrence_time", 
    "notidel", 
    "development_period", 
    "pmt_no",
    "log1_paid_cumulative",
    "max_paid_dev_factor",
    "min_paid_dev_factor",
]
output_field = ["claim_size"]
youtput="claim_size"

In [20]:
X_train,  y_train = df.loc[train, list_of_features].values, df.loc[train, [youtput]].values
X_test,  y_test = df.loc[test, list_of_features].values, df.loc[test, [youtput]].values
## X_test, y_test = df.iloc[test, 1:385].values, df.iloc[test, 385].values
print(X_train.shape, X_test.shape)

(85613, 8) (57228, 8)


In [ ]:
# Saving train and test into csv file
np.savez(datapath + '\slice_localization_tr.npz', features=X_train, labels=y_train)
np.savez(datapath + '\slice_localization_te.npz', features=X_test, labels=y_test)

## Music Year Prediction data split

In [9]:
# Load the data: YearsPrediction
df = pd.read_csv(datapath + '\YearPredictionMSD.txt', header=None)

In [10]:
# YearsPrediction data
data = df.loc[:, 1:]
target = df.loc[:, 0]

data.head(5)

,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,-2.46783,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [11]:
# YearsPrediction split
cut = 463715 # From the website
X_train, y_train = data[:cut].values, target[:cut].values
X_test, y_test = data[cut:].values, target[cut:].values

print(X_train.shape, X_test.shape)


(463715, 90) (51630, 90)


In [21]:
# Saving train and test into npz file
np.savez(datapath + '/datwTestTrainSplit_tr.npz', features=X_train, labels=y_train)
np.savez(datapath + '/datwTestTrainSplit_te.npz', features=X_test, labels=y_test)